In [1]:
import os
import re
import math
import json

In [2]:
# Read data
model_name = 'MCM_C1C5'
exp_name = 'init'
eqn_path = os.path.join(os.getcwd(), '{}_{}.eqn'.format(model_name, exp_name))
with open(eqn_path, 'r') as f:
    raw_lines = f.read().splitlines()
# Create a list of dictionaries describing individual reactions: reactants, products, rate coefficient as a string, comments (optional)
eqs = []
for line in raw_lines:
    if not line.startswith('#') and not line.startswith('{{'):
        if exp_name == 'init' or exp_name == 'emis':
            subline = re.search(r'.*\t\s(.*)\s:\s\t(.*)\s\t;', line).group(1,2) # lines without J comment
            reac, prod = [re.findall(r'[A-Za-z]+[\w]*', s) for s in subline[0].split('=')]
            eqs.append(dict(reac=reac, prod=prod, coef=subline[1]))

#### Generic rate coefficients

In [3]:
# Read MCM generic rate coefficients
gen_eqn_path = os.path.join(os.getcwd(), 'mcm_generic_rate_coeffs.txt')
with open(gen_eqn_path, 'r') as f:
    gen_raw_lines = f.read().splitlines()
gen_eqs = []
for line in gen_raw_lines:
    name, expr = re.search(r'([A-Za-z]+[\w]*)\s=\s(.*)', line).group(1, 2)
    gen_eqs.append(dict(name=name, expr=expr))
# Store the names of generic rate coefficients in a list for later use
gen_cf_names = []
for eq in gen_eqs:
    gen_cf_names.append(eq['name'])

In [4]:
# Parameters
TEMP = 298 # temperature [K]
M = 2.46e+19;
N2 = 1.9434e+19;
O2 = 5.166e+18;
H2O = 2.7e+17;
# Dictionary with constant species values
const_spec = {'M': eval('M'), 'N2': eval('N2'), 'O2': eval('O2'), 'H2O': eval('H2O')}
# Calculate generic rate coefficients
KRO2NO = 2.7e-12*math.exp(360/TEMP)
KRO2HO2 = 2.91e-13*math.exp(1300/TEMP)
KAPHO2 = 5.2e-13*math.exp(980/TEMP)
KAPNO = 7.5e-12*math.exp(290/TEMP)
KRO2NO3 = 2.3e-12
KNO3AL = 1.4e-12*math.exp(-1860/TEMP)
KDEC = 1.00e+06
KROPRIM = 2.50e-14*math.exp(-300/TEMP)
KROSEC = 2.50e-14*math.exp(-300/TEMP)
KCH3O2 = 1.03e-13*math.exp(365/TEMP)
K298CH3O2 = 3.5e-13
K14ISOM1 = 3.00e+07*math.exp(-5300/TEMP)
KD0 = 1.10e-05*M*math.exp(-10100/TEMP)
KDI = 1.90e+17*math.exp(-14100/TEMP)
KRD = KD0/KDI
FCD = 0.30
NCD = 0.75-1.27*(math.log10(FCD))
FD = 10**(math.log10(FCD)/(1+(math.log10(KRD)/NCD)**2))
KBPAN = (KD0*KDI)*FD/(KD0+KDI)
KC0 = 3.28e-28*M*(TEMP/300)**(-6.87)
KCI = 1.125e-11*(TEMP/300)**(-1.105)
KRC = KC0/KCI
FCC = 0.30
NC = 0.75-1.27*(math.log10(FCC))
FC = 10**(math.log10(FCC)/(1+(math.log10(KRC)/NC)**2))
KFPAN = (KC0*KCI)*FC/(KC0+KCI)
K10 = 1.0e-31*M*(TEMP/300)**(-1.6)
K1I = 5.0e-11*(TEMP/300)**(-0.3)
KR1 = K10/K1I
FC1 = 0.85
NC1 = 0.75-1.27*(math.log10(FC1))
F1 = 10**(math.log10(FC1)/(1+(math.log10(KR1)/NC1)**2))
KMT01 = (K10*K1I)*F1/(K10+K1I)
K20 = 1.3e-31*M*(TEMP/300)**(-1.5)
K2I = 2.3e-11*(TEMP/300)**0.24
KR2 = K20/K2I
FC2 = 0.6
NC2 = 0.75-1.27*(math.log10(FC2))
F2 = 10**(math.log10(FC2)/(1+(math.log10(KR2)/NC2)**2))
KMT02 = (K20*K2I)*F2/(K20+K2I)
K30 = 3.6e-30*M*(TEMP/300)**(-4.1)
K3I = 1.9e-12*(TEMP/300)**0.2
KR3 = K30/K3I
FC3 = 0.35
NC3 = 0.75-1.27*(math.log10(FC3))
F3 = 10**(math.log10(FC3)/(1+(math.log10(KR3)/NC3)**2))
KMT03 = (K30*K3I)*F3/(K30+K3I)
K40 = 1.3e-3*M*(TEMP/300)**(-3.5)*math.exp(-11000/TEMP)
K4I = 9.7e+14*(TEMP/300)**0.1*math.exp(-11080/TEMP)
KR4 = K40/K4I
FC4 = 0.35
NC4 = 0.75-1.27*(math.log10(FC4))
F4 = 10**(math.log10(FC4)/(1+(math.log10(KR4)/NC4)**2))
KMT04 = (K40*K4I)*F4/(K40+K4I)
KMT05 = 1.44e-13*(1+(M/4.2e+19))
KMT06 = 1 + (1.40e-21*math.exp(2200/TEMP)*H2O)
K70 = 7.4e-31*M*(TEMP/300)**(-2.4)
K7I = 3.3e-11*(TEMP/300)**(-0.3)
KR7 = K70/K7I
FC7 = 0.81
NC7 = 0.75-1.27*(math.log10(FC7))
F7 = 10**(math.log10(FC7)/(1+(math.log10(KR7)/NC7)**2))
KMT07 = (K70*K7I)*F7/(K70+K7I)
K80 = 3.2e-30*M*(TEMP/300)**(-4.5)
K8I = 3.0e-11
KR8 = K80/K8I
FC8 = 0.41
NC8 = 0.75-1.27*(math.log10(FC8))
F8 = 10**(math.log10(FC8)/(1+(math.log10(KR8)/NC8)**2))
KMT08 = (K80*K8I)*F8/(K80+K8I)
K90 = 1.4e-31*M*(TEMP/300)**(-3.1)
K9I = 4.0e-12
KR9 = K90/K9I
FC9 = 0.4
NC9 = 0.75-1.27*(math.log10(FC9))
F9 = 10**(math.log10(FC9)/(1+(math.log10(KR9)/NC9)**2))
KMT09 = (K90*K9I)*F9/(K90+K9I)
K100 = 4.10e-05*M*math.exp(-10650/TEMP)
K10I = 6.0e+15*math.exp(-11170/TEMP)
KR10 = K100/K10I
FC10 = 0.4
NC10 = 0.75-1.27*(math.log10(FC10))
F10 = 10**(math.log10(FC10)/(1+(math.log10(KR10)/NC10)**2))
KMT10 = (K100*K10I)*F10/(K100+K10I)
K1 = 2.40e-14*math.exp(460/TEMP)
K3 = 6.50e-34*math.exp(1335/TEMP)
K4 = 2.70e-17*math.exp(2199/TEMP)
K2 = (K3*M)/(1+(K3*M/K4))
KMT11 = K1 + K2
K120 = 2.5e-31*M*(TEMP/300)**(-2.6)
K12I = 2.0e-12
KR12 = K120/K12I
FC12 = 0.53
NC12 = 0.75-1.27*(math.log10(FC12))
F12 = 10**(math.log10(FC12)/(1.0+(math.log10(KR12)/NC12)**2))
KMT12 = (K120*K12I*F12)/(K120+K12I)
K130 = 2.5e-30*M*(TEMP/300)**(-5.5)
K13I = 1.8e-11
KR13 = K130/K13I
FC13 = 0.36
NC13 = 0.75-1.27*(math.log10(FC13))
F13 = 10**(math.log10(FC13)/(1+(math.log10(KR13)/NC13)**2))
KMT13 = (K130*K13I)*F13/(K130+K13I)
K140 = 9.0e-5*math.exp(-9690/TEMP)*M
K14I = 1.1e+16*math.exp(-10560/TEMP)
KR14 = K140/K14I
FC14 = 0.36
NC14 = 0.75-1.27*(math.log10(FC14))
F14 = 10**(math.log10(FC14)/(1+(math.log10(KR14)/NC14)**2))
KMT14 = (K140*K14I)*F14/(K140+K14I)
K150 = 8.6e-29*M*(TEMP/300)**(-3.1)
K15I = 9.0e-12*(TEMP/300)**(-0.85)
KR15 = K150/K15I
FC15 = 0.48
NC15 = 0.75-1.27*(math.log10(FC15))
F15 = 10**(math.log10(FC15)/(1+(math.log10(KR15)/NC15)**2))
KMT15 = (K150*K15I)*F15/(K150+K15I)
K160 = 8e-27*M*(TEMP/300)**(-3.5)
K16I = 3.0e-11*(TEMP/300)**(-1)
KR16 = K160/K16I
FC16 = 0.5
NC16 = 0.75-1.27*(math.log10(FC16))
F16 = 10**(math.log10(FC16)/(1+(math.log10(KR16)/NC16)**2))
KMT16 = (K160*K16I)*F16/(K160+K16I)
K170 = 5.0e-30*M*(TEMP/300)**(-1.5)
K17I = 1.0e-12
KR17 = K170/K17I
FC17 = 0.17*math.exp(-51/TEMP)+math.exp(-TEMP/204)
NC17 = 0.75-1.27*(math.log10(FC17))
F17 = 10**(math.log10(FC17)/(1.0+(math.log10(KR17)/NC17)**2))
KMT17 = (K170*K17I*F17)/(K170+K17I)
KMT18 = 9.5e-39*O2*math.exp(5270/TEMP)/(1+7.5e-29*O2*math.exp(5610/TEMP))
KPPN0 = 1.7e-03*math.exp(-11280/TEMP)*M
KPPNI = 8.3e+16*math.exp(-13940/TEMP)
KRPPN = KPPN0/KPPNI
FCPPN = 0.36
NCPPN = 0.75-1.27*(math.log10(FCPPN))
FPPN = 10**(math.log10(FCPPN)/(1+(math.log10(KRPPN)/NCPPN)**2))
KBPPN = (KPPN0*KPPNI)*FCPPN/(KPPN0+KPPNI)

In [5]:
# Add generic rate coefficients to a dictionary
gen_eqs[0].setdefault('num', []).append(KRO2NO)
gen_eqs[1].setdefault('num', []).append(KRO2HO2)
gen_eqs[2].setdefault('num', []).append(KAPHO2)
gen_eqs[3].setdefault('num', []).append(KAPNO)
gen_eqs[4].setdefault('num', []).append(KRO2NO3)
gen_eqs[5].setdefault('num', []).append(KNO3AL)
gen_eqs[6].setdefault('num', []).append(KDEC)
gen_eqs[7].setdefault('num', []).append(KROPRIM)
gen_eqs[8].setdefault('num', []).append(KROSEC)
gen_eqs[9].setdefault('num', []).append(KCH3O2)
gen_eqs[10].setdefault('num', []).append(K298CH3O2)
gen_eqs[11].setdefault('num', []).append(K14ISOM1)
gen_eqs[12].setdefault('num', []).append(KD0)
gen_eqs[13].setdefault('num', []).append(KDI)
gen_eqs[14].setdefault('num', []).append(KRD)
gen_eqs[15].setdefault('num', []).append(FCD)
gen_eqs[16].setdefault('num', []).append(NCD)
gen_eqs[17].setdefault('num', []).append(FD)
gen_eqs[18].setdefault('num', []).append(KBPAN)
gen_eqs[19].setdefault('num', []).append(KC0)
gen_eqs[20].setdefault('num', []).append(KCI)
gen_eqs[21].setdefault('num', []).append(KRC)
gen_eqs[22].setdefault('num', []).append(FCC)
gen_eqs[23].setdefault('num', []).append(NC)
gen_eqs[24].setdefault('num', []).append(FC)
gen_eqs[25].setdefault('num', []).append(KFPAN)
gen_eqs[26].setdefault('num', []).append(K10)
gen_eqs[27].setdefault('num', []).append(K1I)
gen_eqs[28].setdefault('num', []).append(KR1)
gen_eqs[29].setdefault('num', []).append(FC1)
gen_eqs[30].setdefault('num', []).append(NC1)
gen_eqs[31].setdefault('num', []).append(F1)
gen_eqs[32].setdefault('num', []).append(KMT01)
gen_eqs[33].setdefault('num', []).append(K20)
gen_eqs[34].setdefault('num', []).append(K2I)
gen_eqs[35].setdefault('num', []).append(KR2)
gen_eqs[36].setdefault('num', []).append(FC2)
gen_eqs[37].setdefault('num', []).append(NC2)
gen_eqs[38].setdefault('num', []).append(F2)
gen_eqs[39].setdefault('num', []).append(KMT02)
gen_eqs[40].setdefault('num', []).append(K30)
gen_eqs[41].setdefault('num', []).append(K3I)
gen_eqs[42].setdefault('num', []).append(KR3)
gen_eqs[43].setdefault('num', []).append(FC3)
gen_eqs[44].setdefault('num', []).append(NC3)
gen_eqs[45].setdefault('num', []).append(F3)
gen_eqs[46].setdefault('num', []).append(KMT03)
gen_eqs[47].setdefault('num', []).append(K40)
gen_eqs[48].setdefault('num', []).append(K4I)
gen_eqs[49].setdefault('num', []).append(KR4)
gen_eqs[50].setdefault('num', []).append(FC4)
gen_eqs[51].setdefault('num', []).append(NC4)
gen_eqs[52].setdefault('num', []).append(F4)
gen_eqs[53].setdefault('num', []).append(KMT04)
gen_eqs[54].setdefault('num', []).append(KMT05)
gen_eqs[55].setdefault('num', []).append(KMT06)
gen_eqs[56].setdefault('num', []).append(K70)
gen_eqs[57].setdefault('num', []).append(K7I)
gen_eqs[58].setdefault('num', []).append(KR7)
gen_eqs[59].setdefault('num', []).append(FC7)
gen_eqs[60].setdefault('num', []).append(NC7)
gen_eqs[61].setdefault('num', []).append(F7)
gen_eqs[62].setdefault('num', []).append(KMT07)
gen_eqs[63].setdefault('num', []).append(K80)
gen_eqs[64].setdefault('num', []).append(K8I)
gen_eqs[65].setdefault('num', []).append(KR8)
gen_eqs[66].setdefault('num', []).append(FC8)
gen_eqs[67].setdefault('num', []).append(NC8)
gen_eqs[68].setdefault('num', []).append(F8)
gen_eqs[69].setdefault('num', []).append(KMT08)
gen_eqs[70].setdefault('num', []).append(K90)
gen_eqs[71].setdefault('num', []).append(K9I)
gen_eqs[72].setdefault('num', []).append(KR9)
gen_eqs[73].setdefault('num', []).append(FC9)
gen_eqs[74].setdefault('num', []).append(NC9)
gen_eqs[75].setdefault('num', []).append(F9)
gen_eqs[76].setdefault('num', []).append(KMT09)
gen_eqs[77].setdefault('num', []).append(K100)
gen_eqs[78].setdefault('num', []).append(K10I)
gen_eqs[79].setdefault('num', []).append(KR10)
gen_eqs[80].setdefault('num', []).append(FC10)
gen_eqs[81].setdefault('num', []).append(NC10)
gen_eqs[82].setdefault('num', []).append(F10)
gen_eqs[83].setdefault('num', []).append(KMT10)
gen_eqs[84].setdefault('num', []).append(K1)
gen_eqs[85].setdefault('num', []).append(K3)
gen_eqs[86].setdefault('num', []).append(K4)
gen_eqs[87].setdefault('num', []).append(K2)
gen_eqs[88].setdefault('num', []).append(KMT11)
gen_eqs[89].setdefault('num', []).append(K120)
gen_eqs[90].setdefault('num', []).append(K12I)
gen_eqs[91].setdefault('num', []).append(KR12)
gen_eqs[92].setdefault('num', []).append(FC12)
gen_eqs[93].setdefault('num', []).append(NC12)
gen_eqs[94].setdefault('num', []).append(F12)
gen_eqs[95].setdefault('num', []).append(KMT12)
gen_eqs[96].setdefault('num', []).append(K130)
gen_eqs[97].setdefault('num', []).append(K13I)
gen_eqs[98].setdefault('num', []).append(KR13)
gen_eqs[99].setdefault('num', []).append(FC13)
gen_eqs[100].setdefault('num', []).append(NC13)
gen_eqs[101].setdefault('num', []).append(F13)
gen_eqs[102].setdefault('num', []).append(KMT13)
gen_eqs[103].setdefault('num', []).append(K140)
gen_eqs[104].setdefault('num', []).append(K14I)
gen_eqs[105].setdefault('num', []).append(KR14)
gen_eqs[106].setdefault('num', []).append(FC14)
gen_eqs[107].setdefault('num', []).append(NC14)
gen_eqs[108].setdefault('num', []).append(F14)
gen_eqs[109].setdefault('num', []).append(KMT14)
gen_eqs[110].setdefault('num', []).append(K150)
gen_eqs[111].setdefault('num', []).append(K15I)
gen_eqs[112].setdefault('num', []).append(KR15)
gen_eqs[113].setdefault('num', []).append(FC15)
gen_eqs[114].setdefault('num', []).append(NC15)
gen_eqs[115].setdefault('num', []).append(F15)
gen_eqs[116].setdefault('num', []).append(KMT15)
gen_eqs[117].setdefault('num', []).append(K160)
gen_eqs[118].setdefault('num', []).append(K16I)
gen_eqs[119].setdefault('num', []).append(KR16)
gen_eqs[120].setdefault('num', []).append(FC16)
gen_eqs[121].setdefault('num', []).append(NC16)
gen_eqs[122].setdefault('num', []).append(F16)
gen_eqs[123].setdefault('num', []).append(KMT16)
gen_eqs[124].setdefault('num', []).append(K170)
gen_eqs[125].setdefault('num', []).append(K17I)
gen_eqs[126].setdefault('num', []).append(KR17)
gen_eqs[127].setdefault('num', []).append(FC17)
gen_eqs[128].setdefault('num', []).append(NC17)
gen_eqs[129].setdefault('num', []).append(F17)
gen_eqs[130].setdefault('num', []).append(KMT17)
gen_eqs[131].setdefault('num', []).append(KMT18)
gen_eqs[132].setdefault('num', []).append(KPPN0)
gen_eqs[133].setdefault('num', []).append(KPPNI)
gen_eqs[134].setdefault('num', []).append(KRPPN)
gen_eqs[135].setdefault('num', []).append(FCPPN)
gen_eqs[136].setdefault('num', []).append(NCPPN)
gen_eqs[137].setdefault('num', []).append(FPPN)
gen_eqs[138].setdefault('num', []).append(KBPPN)

In [6]:
# Add info about generic rate coefficients to reaction dictionaries
for geq in gen_eqs:
    for eq in eqs:
        if geq['name'] == eq['coef']:
            eq['num'] = geq['num'][0]

#### Calculate rate coefficients in case of varying photolysis

In [7]:
#----------Old code
flt_re = r'-?[0-9]+\.*[0-9]*' # regular expression for float and integer number
# Dictionary with constant species values
const_spec = {'M': eval('M'), 'N2': eval('N2'), 'O2': eval('O2'), 'H2O': eval('H2O')}
# Main loop
if exp_name == 'init':
    for eq in eqs:
        if len(list(eq.keys())) == 3:
            if 'EXP' not in eq['coef']:
                if 'TEMP' in eq['coef']:
                    eq['num'] = 5.6e-34*N2*(TEMP/300)**(-2.6)*O2+6.0e-34*O2*(TEMP/300)**(-2.6)*O2
                else:
                    if '*' not in eq['coef']:
                        if 'J' not in eq['coef']:
                            # 5.5D-16
                            ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)'.format(flt_re=flt_re), eq['coef'])[0]
                            eq['num'] = float(re.sub(r'[DdEe]', r'e', ptn))
                        else:
                            # J(54)+J(15)
                            eq['num'] = 1
                    else:
                        if len(eq['coef'].split('*')) == 2:
                            if eq['coef'].split('*')[0] in const_spec.keys() or eq['coef'].split('*')[1] in const_spec.keys():
                                if eq['coef'].split('*')[0] in gen_cf_names or eq['coef'].split('*')[1] in gen_cf_names:
                                    for i in gen_eqs:
                                        for j in const_spec.keys():
                                            # KROPRIM*O2
                                            if eq['coef'].split('*')[0] == i['name']  and eq['coef'].split('*')[1] == j:
                                                eq['num'] = i['num'][0] * const_spec[j]
                                else:
                                    # 2.14D-10*H2O
                                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*(\w+)'.format(flt_re=flt_re), eq['coef'])[0]
                                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                                    eq['num'] = p1 * const_spec[ptn[1]]
                            else:
                                if eq['coef'].split('*')[0] in gen_cf_names or eq['coef'].split('*')[1] in gen_cf_names:
                                    for i in gen_eqs:
                                        # KRO2HO2*0.520
                                        if eq['coef'].split('*')[0] == i['name']:
                                            eq['num'] = i['num'][0] * float(eq['coef'].split('*')[1])
                                elif 'J' in eq['coef']:
                                    # J(17)*2.0
                                    eq['num'] = 1
                                elif 'RO2' in eq['coef']:
                                    # 1.00D-11*RO2
                                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)'.format(flt_re=flt_re), eq['coef'].split('*')[0])[0]
                                    eq['num'] = float(re.sub(r'[DdEe]', r'e', ptn))
                                else:
                                    # 1.43D-10*0.564
                                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)'.format(flt_re=flt_re), eq['coef'].split('*')[0])[0]
                                    p1 = float(re.sub(r'[DdEe]', r'e', ptn))
                                    eq['num'] = p1 * float(eq['coef'].split('*')[1])
                        else:
                            if 'RO2' not in eq['coef']:
                                # 2*KNO3AL*4.0
                                for i in gen_eqs:
                                    if i['name'] == 'KNO3AL':
                                        eq['num'] = 2*i['num'][0]*4.0
                            else:
                                if len(eq['coef'].split('*')) == 3:
                                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)'.format(flt_re=flt_re), eq['coef'].split('*')[0])[0]
                                    p1 = float(re.sub(r'[DdEe]', r'e', ptn))
                                    # 1.00D-11*RO2*0.7
                                    if eq['coef'].split('*')[1] == 'RO2':
                                        eq['num'] = p1 * float(eq['coef'].split('*')[2])
                                    else:
                                        # 1.30D-12*0.2*RO2
                                        eq['num'] = p1 * float(eq['coef'].split('*')[1])
                                else:
                                    # 2*(K298CH3O2*3D-13)**0.5*RO2*0.2
                                    ptn = re.findall(r'({flt_re})\*\((\w+)\*({flt_re}[DdEe] *[\+-?]*[0-9]+).*\*({flt_re})\*RO2\*({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                                    p1 = float(ptn[0])
                                    p3 = float(re.sub(r'[DdEe]', r'e', ptn[2]))
                                    p4, p5 = map(float, ptn[3:])
                                    for i in gen_eqs:
                                        if i['name'] == ptn[1]:
                                            eq['num'] = p1 * (i['num'][0] * p3)**p4 * p5
            else:
                if 'RO2' in eq['coef']:
                    if eq['coef'].count('TEMP') == 2:
                        # 2*(KCH3O2*6.4D-14*(TEMP/300)**0*EXP(0/TEMP))**0.5*RO2*0.6
                        ptn = re.findall(r'({flt_re})\*\((\w+)\*({flt_re}[DdEe] *[\+-?]*[0-9]+).*\/({flt_re}).*({flt_re})\*EXP.*({flt_re})\/TEMP.*\*({flt_re})\*RO2\*({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                        p1 = float(ptn[0])
                        p3 = float(re.sub(r'[DdEe]', r'e', ptn[2]))
                        p4, p5, p6, p7, p8 = map(float, ptn[3:])
                        for i in gen_eqs:
                            if ptn[1] == i['name']:
                                eq['num'] = p1 * (i['num'][0] * p3 * (TEMP/p4)**p5 * math.exp(p6/TEMP))**p7 * p8
                    elif eq['coef'].count('TEMP') == 1 and '**' in eq['coef']:
                        # 2*(KCH3O2*1.6D-12*EXP(-2200/TEMP))**0.5*RO2*0.2
                        ptn = re.findall(r'({flt_re})\*\((\w+)\*({flt_re}[DdEe] *[\+-?]*[0-9]+).*\(({flt_re})\/TEMP.*\*({flt_re})\*RO2\*({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                        p1 = float(ptn[0])
                        p3 = float(re.sub(r'[DdEe]', r'e', ptn[2]))
                        p4, p5, p6 = map(float, ptn[3:])
                        for i in gen_eqs:
                            if ptn[1] == i['name']:
                                eq['num'] = p1 * (i['num'][0] * p3 * math.exp(p4/TEMP))**p5 * p6
                    elif '1-' in eq['coef']:
                        # 2*KCH3O2*RO2*0.5*(1-7.18*EXP(-885/TEMP))
                        ptn = re.findall(r'({flt_re})\*(\w+)\*RO2\*({flt_re})\*\(({flt_re})-({flt_re}).*EXP\(({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                        p1 = float(ptn[0])
                        p3, p4, p5, p6 = map(float, ptn[2:])
                        for i in gen_eqs:
                            if ptn[1] == i['name']:
                                eq['num'] = p1 * i['num'][0] * p3 * (p4 - p5 * math.exp(p6/TEMP))
                    else:
                        # 2*KCH3O2*RO2*7.18*EXP(-885/TEMP)
                        ptn = re.findall(r'({flt_re})\*(\w+)\*RO2\*({flt_re}).*EXP\(({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                        p1 = float(ptn[0])
                        p3, p4 = map(float, ptn[2:])
                        for i in gen_eqs:
                            if ptn[1] == i['name']:
                                eq['num'] = p1 * i['num'][0] * p3 * math.exp(p4/TEMP)
                else:
                    if eq['coef'].count('TEMP') == 2:
                        if 'KMT06' in eq['coef']:
                            # 2.20D-13*KMT06*EXP(600/TEMP)+1.90D-33*M*KMT06*EXP(980/TEMP)
                            for i in gen_eqs:
                                if i['name'] == 'KMT06':
                                    eq['num'] = 2.20e-13 * i['num'][0] * math.exp(600/TEMP) + 1.90e-33 * M * i['num'][0] * math.exp(980/TEMP)
                        elif eq['coef'] == '3.2D-11*EXP(67/TEMP)*O2+2.0D-11*EXP(130/TEMP)*N2':
                            eq['num'] = 3.2e-11 * math.exp(67/TEMP) * O2 + 2.0e-11 * math.exp(130/TEMP) * N2 # big number
                        elif eq['coef'] == '2.03D-16*(TEMP/300)**4.57*EXP(693/TEMP)':
                            eq['num'] = 2.03e-16 * (TEMP/300)**4.57 * math.exp(693/TEMP)
                        elif eq['coef'] == '3.8D-13*EXP(780/TEMP)*(1-1/(1+498*EXP(-1160/TEMP)))':
                            eq['num'] = 3.8e-13 * math.exp(780/TEMP) * (1 - 1/(1 + 498 * math.exp(-1160/TEMP)))
                        elif eq['coef'] == '3.8D-13*EXP(780/TEMP)*(1/(1+498*EXP(-1160/TEMP)))':
                            eq['num'] = 3.8e-13 * math.exp(780/TEMP) * (1/(1 + 498 * math.exp(-1160/TEMP)))
                        elif eq['coef'] == '8.8D-12*EXP(-1320/TEMP) + 1.7D-14*EXP(423/TEMP)':
                            eq['num'] = 8.8e-12 * math.exp(-1320/TEMP) + 1.7e-14 * math.exp(423/TEMP)
                        elif eq['coef'] == '1.40D-18*TEMP**2*EXP(194/TEMP)':
                            eq['num'] = 1.40e-18 * TEMP**2 * math.exp(194/TEMP)
                        else:
                            # 1.16D-17*TEMP**2*EXP(225/TEMP)*0.206
                            ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+).*\*({flt_re})\*EXP\(({flt_re})\/TEMP\)\*({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                            p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                            p2, p3, p4 = map(float, ptn[1:])
                            eq['num'] = p1 * TEMP**p2 * math.exp(p3/TEMP) * p4
                    else:
                        if len(eq['coef'].split('*')) == 3:
                            if eq['coef'].split('*')[2] in const_spec.keys():
                                if eq['coef'] == '7.00D11*EXP(-3160/TEMP)+5.00D-12*O2':
                                    eq['num'] = 7.00e+11 * math.exp(-3160/TEMP) + 5.00e-12 * O2 # big number
                                else:
                                    # 3.3D-39*EXP(530/TEMP)*O2
                                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*EXP\(({flt_re}).*TEMP\)\*(\w+)'.format(flt_re=flt_re), eq['coef'])[0]
                                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                                    p2 = float(ptn[1])
                                    eq['num'] = p1 * math.exp(p2/TEMP) * const_spec[ptn[2]]
                            else:
                                try:
                                    # 3.2D-13*EXP(690/TEMP)*1.0
                                    p3 = float(eq['coef'].split('*')[2])
                                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*EXP\(({flt_re}).*\*({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                                    p2 = float(ptn[1])
                                    eq['num'] = p1 * math.exp(p2/TEMP) * p3
                                except ValueError:
                                    # 1.4D-10*0.43*EXP(75/TEMP)
                                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*({flt_re})\*EXP\(({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                                    p2, p3 = map(float, ptn[1:])
                                    eq['num'] = p1 * p2 * math.exp(p3/TEMP)
                        else:
                            if eq['coef'] == '5.00D-12*O2*3.2*(1-EXP(-550/TEMP))':
                                eq['num'] = 5.00e-12 * O2 * 3.2 * (1 - math.exp(-550/TEMP)) # big number
                            elif eq['coef'] == '5.00D-12*O2*3.2*EXP(-550/TEMP)':
                                eq['num'] = 5.00e-12 * O2 * 3.2 * math.exp(-550/TEMP) # big number
                            else:
                                # 8.0D-12*EXP(-2060/TEMP)
                                ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*EXP\(({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                                p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                                p2 = float(ptn[1])
                                eq['num'] = p1 * math.exp(p2/TEMP)

#### Calculate rate coefficient in case of constant midday photolysis

In [8]:
flt_re = r'-?[0-9]+\.*[0-9]*' # regular expression for float and integer number
# Load midday photolysis rate coefficients
if exp_name == 'stat':
    j_expr_path = os.path.join(os.getcwd(), 'mcm_photolysis_midday.txt')
    with open(j_expr_path, 'r') as f:
        j_raw_lines = f.read().splitlines()
    j_num = {}
    for line in j_raw_lines[1::]:
        pattern_j = re.findall(r'(J\(\d+\)) ({flt_re}[Ee] *[\+-?]*[0-9]+)'.format(flt_re=flt_re), line)
        j_name = pattern_j[0][0]
        j_coef = float(pattern_j[0][1])
        j_num[j_name] = j_coef
# Handling uprocessed rate coefficients
class UnprocessedRateCoef(Exception):
    pass
# Calculate other rate coefficients
if exp_name == 'stat':
    for eq in eqs:
        if 'num' not in eq.keys():
            if 'comm' in eq.keys():
                if 'J' in eq['comm']:
                    # J(54)+J(15)
                    if len(eq['comm'].split('+')) == 2:
                        eq['num'] = j_num[eq['comm'].split('+')[0]] + j_num[eq['comm'].split('+')[1]]
                    # J(17)*2.0
                    elif len(eq['comm'].split('*')) == 2:
                        eq['num'] = j_num[eq['comm'].split('*')[0]] * float(eq['comm'].split('*')[1])
                    # J(1)
                    elif len(re.findall(r'^J\(\d+\)$', eq['comm'])) == 1:
                        eq['num'] = j_num[eq['comm']]
                    else:
                        raise UnprocessedRateCoef('Unprocessed photolysis reaction is found.')
            elif 'EXP' not in eq['coef']:
                if '5.6D-34*N2*(TEMP/300)**(-2.6)*O2+6.0D-34*O2*(TEMP/300)**(-2.6)*O2' in eq['coef']:
                    eq['num'] = 5.6e-34*N2*(TEMP/300)**(-2.6)*O2+6.0e-34*O2*(TEMP/300)**(-2.6)*O2
                # 1.00D-11*0.200, 1.00D-11*RO2, 2.14D-10*H2O, KRO2NO*0.005, KROPRIM*O2
                elif len(eq['coef'].split('*')) == 2:
                    one = eq['coef'].split('*')[0] # sci number or generic rate coefficient
                    two = eq['coef'].split('*')[1] # float number or number density of a constant species or RO2
                    # 1.00D-11*0.200, 1.00D-11*RO2, 2.14D-10*H2O
                    if len(re.findall('({flt_re}[Dd][\+-?]*[0-9]+)'.format(flt_re=flt_re), one)) == 1:
                        # 1.00D-11*0.200
                        if two not in const_spec.keys() and two != 'RO2':
                            eq['num'] = float(re.sub(r'[DdE]', r'e', one)) * float(two)
                        # 1.00D-11*RO2
                        elif two not in const_spec.keys() and two == 'RO2':
                            eq['num'] = float(re.sub(r'[DdE]', r'e', one))
                        # 2.14D-10*H2O
                        elif two in const_spec.keys():
                            eq['num'] = float(re.sub(r'[DdE]', r'e', one)) * const_spec[two]
                        else:
                            raise UnprocessedRateCoef('Unprocessed reaction is found.')
                    # KRO2NO*0.005, KROPRIM*O2
                    elif one in gen_cf_names:
                        # KRO2NO*0.005
                        if two not in const_spec.keys():
                            eq['num'] = eval(one) * float(two)
                        # KROPRIM*O2
                        elif two in const_spec.keys():
                            eq['num'] = eval(one) * const_spec[two]
                        else:
                            raise UnprocessedRateCoef('Unprocessed reaction is found.')
                    else:
                        raise UnprocessedRateCoef('Unprocessed reaction is found.')
                # 2.00D-12*0.2*RO2
                elif len(re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*({flt_re})\*RO2'.format(flt_re=flt_re), eq['coef'])) == 1:
                    ptn = re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*({flt_re})\*RO2'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdE]', r'e', ptn[0]))
                    p2 = float(ptn[1])
                    eq['num'] = p1 * p2
                # 1.00D-11*RO2*0.7
                elif len(re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*RO2\*({flt_re})'.format(flt_re=flt_re), eq['coef'])) == 1:
                    ptn = re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*RO2\*({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdE]', r'e', ptn[0]))
                    p2 = float(ptn[1])
                    eq['num'] = p1 * p2
                # 2.0D-11
                elif len(re.findall('^({flt_re}[Dd][\+-?]*[0-9]+)$'.format(flt_re=flt_re), eq['coef'])) == 1:
                    eq['num'] = float(re.sub(r'[DdE]', r'e', eq['coef']))
                # 2*(K298CH3O2*3D-13)**0.5*RO2*0.2
                elif len(re.findall(r'({flt_re})\*\((\w+)\*({flt_re}[Dd][\+-?]*[0-9]+)\)\*\*({flt_re})\*RO2\*({flt_re})'.format(flt_re=flt_re), 
                                    eq['coef'])) == 1:
                    ptn = re.findall(r'({flt_re})\*\((\w+)\*({flt_re}[Dd][\+-?]*[0-9]+)\)\*\*({flt_re})\*RO2\*({flt_re})'.format(flt_re=flt_re), 
                                    eq['coef'])[0]
                    p1 = float(ptn[0])
                    p3 = float(re.sub(r'[DdE]', r'e', ptn[2]))
                    p4, p5 = map(float, ptn[3:])
                    eq['num'] = p1 * (eval(ptn[1]) * p3)**p4 * p5
                elif eq['coef'] == '2*KNO3AL*4.0':
                    eq['num'] = 2 * eval('KNO3AL') * 4.0
                else:
                    print(eq)
                    raise UnprocessedRateCoef('Unprocessed reaction is found.')
            elif 'EXP' in eq['coef']:
                if eq['coef'].count('TEMP') == 2:
                    # 2*(KCH3O2*6.4D-14*(TEMP/300)**0*EXP(0/TEMP))**0.5*RO2*0.6
                    try:
                        ptn = re.findall(r'({flt_re})\*\((\w+)\*({flt_re}[DdEe] *[\+-?]*[0-9]+).*\/({flt_re}).*({flt_re})\*EXP.*({flt_re})\/TEMP.*\*({flt_re})\*RO2\*({flt_re})'.format(flt_re=flt_re), 
                                         eq['coef'])[0]
                        p1 = float(ptn[0])
                        p3 = float(re.sub(r'[DdE]', r'e', ptn[2]))
                        p4, p5, p6, p7, p8 = map(float, ptn[3:])
                        eq['num'] = p1 * (eval(ptn[1]) * p3 * (TEMP/p4)**p5 * math.exp(p6/TEMP))**p7 * p8
                    except IndexError:
                        # 1.16D-17*TEMP**2*EXP(225/TEMP)*0.206
                        try:
                            ptn = re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*TEMP\*\*({flt_re})\*EXP\(({flt_re})\/TEMP\)\*({flt_re})'.format(flt_re=flt_re), 
                                             eq['coef'])[0]
                            p1 = float(re.sub(r'[DdE]', r'e', ptn[0]))
                            p2, p3, p4 = map(float, ptn[1:])
                            eq['num'] = p1 * TEMP**p2 * math.exp(p3/TEMP) * p4
                        except IndexError:
                            if eq['coef'] == '3.2D-11*EXP(67/TEMP)*O2+2.0D-11*EXP(130/TEMP)*N2':
                                eq['num'] = 3.2e-11 * math.exp(67/TEMP) * O2 + 2.0e-11 * math.exp(130/TEMP) * N2
                            elif eq['coef'] == '2.03D-16*(TEMP/300)**4.57*EXP(693/TEMP)':
                                eq['num'] = 2.03e-16 * (TEMP/300)**4.57 * math.exp(693/TEMP)
                            elif eq['coef'] == '2.20D-13*KMT06*EXP(600/TEMP)+1.90D-33*M*KMT06*EXP(980/TEMP)':
                                eq['num'] = 2.20e-13 * eval('KMT06') * math.exp(600/TEMP) + 1.90e-33 * M * eval('KMT06') * math.exp(980/TEMP)
                            elif eq['coef'] == '3.8D-13*EXP(780/TEMP)*(1-1/(1+498*EXP(-1160/TEMP)))':
                                eq['num'] = 3.8e-13 * math.exp(780/TEMP) * (1 - 1/(1 + 498 * math.exp(-1160/TEMP)))
                            elif eq['coef'] == '3.8D-13*EXP(780/TEMP)*(1/(1+498*EXP(-1160/TEMP)))':
                                eq['num'] = 3.8e-13 * math.exp(780/TEMP) * (1/(1 + 498 * math.exp(-1160/TEMP)))
                            elif eq['coef'] == '8.8D-12*EXP(-1320/TEMP) + 1.7D-14*EXP(423/TEMP)':
                                eq['num'] = 8.8e-12 * math.exp(-1320/TEMP) + 1.7e-14 * math.exp(423/TEMP)
                            elif eq['coef'] == '1.40D-18*TEMP**2*EXP(194/TEMP)':
                                eq['num'] = 1.40e-18 * TEMP**2 * math.exp(194/TEMP)
                            else:
                                raise UnprocessedRateCoef('Unprocessed reaction is found.')
                elif 'RO2' in eq['coef']:
                    # 2*KCH3O2*RO2*0.5*(1-7.18*EXP(-885/TEMP))
                    try:
                        ptn = re.findall(r'(\d+)\*(\w+)\*RO2\*({flt_re})\*\((\d)-({flt_re})\*EXP\(({flt_re})\/TEMP'.format(flt_re=flt_re), 
                                             eq['coef'])[0] # 6 parameters
                        p1 = float(ptn[0])
                        p3, p4, p5, p6 = map(float, ptn[2:])
                        eq['num'] = p1 * eval(ptn[1]) * p3 * (p4 - p5 * math.exp(p6/TEMP))
                    except IndexError:
                        # 2*(KCH3O2*7.8D-14*EXP(1000/TEMP))**0.5*RO2*0.2
                        try:
                            ptn = re.findall(r'(\d+)\*\((\w+)\*({flt_re}[Dd][\+-?]*[0-9]+)\*EXP\(({flt_re})\/TEMP\)\)\*\*({flt_re})\*RO2\*({flt_re})'.format(flt_re=flt_re), 
                                             eq['coef'])[0] # 6 parameters
                            p1 = float(ptn[0])
                            p3 = float(re.sub(r'[DdE]', r'e', ptn[2]))
                            p4, p5, p6 = map(float, ptn[3:])
                            eq['num'] = p1 * (eval(ptn[1]) * p3 * math.exp(p4/TEMP))**p5 * p6
                        except IndexError:
                            # 2*KCH3O2*RO2*7.18*EXP(-885/TEMP)
                            try:
                                ptn = re.findall(r'(\d+)\*(\w+)\*RO2\*({flt_re})\*EXP\(({flt_re})\/TEMP\)'.format(flt_re=flt_re), 
                                                 eq['coef'])[0] # 4 parameters
                                p1 = float(ptn[0])
                                p3, p4 = map(float, ptn[2:])
                                eq['num'] = p1 * eval(ptn[1]) * p3 * math.exp(p4/TEMP)
                            except IndexError:
                                raise UnprocessedRateCoef('Unprocessed reaction is found.')
                # 8.0D-12*EXP(-2060/TEMP)
                elif len(re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*EXP\(({flt_re})\/TEMP\)$'.format(flt_re=flt_re), eq['coef'])) != 0:
                    ptn = re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*EXP\(({flt_re})\/TEMP\)$'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdE]', r'e', ptn[0]))
                    p2 = float(ptn[1])
                    eq['num'] = p1 * math.exp(p2/TEMP)
                # 3.2D-13*EXP(690/TEMP)*1.0
                elif len(re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*EXP\(({flt_re})\/TEMP\)\*({flt_re})'.format(flt_re=flt_re), eq['coef'])) != 0:
                    ptn = re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*EXP\(({flt_re})\/TEMP\)\*({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdE]', r'e', ptn[0]))
                    p2, p3 = map(float, ptn[1:])
                    eq['num'] = p1 * math.exp(p2/TEMP) * p3
                # 3.3D-39*EXP(530/TEMP)*O2
                elif len(re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*EXP\(({flt_re})\/TEMP\)\*(\w+)'.format(flt_re=flt_re), eq['coef'])) != 0:
                    ptn = re.findall(r'({flt_re}[Dd][\+-?]*[0-9]+)\*EXP\(({flt_re})\/TEMP\)\*(\w+)'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdE]', r'e', ptn[0]))
                    p2 = float(ptn[1])
                    eq['num'] = p1 * math.exp(p2/TEMP) * const_spec[ptn[2]]
                elif '7.00D11*EXP(-3160/TEMP)+5.00D-12*O2' in eq['coef']:
                    eq['num'] = 7.00e11 * math.exp(-3160/TEMP) + 5.00e-12 * O2
                elif '5.00D-12*O2*3.2*(1-EXP(-550/TEMP))' in eq['coef']:
                    eq['num'] = 5.00e-12 * O2 * 3.2 * (1 - math.exp(-550/TEMP))
                elif '5.00D-12*O2*3.2*EXP(-550/TEMP)' in eq['coef']:
                    eq['num'] = 5.00e-12 * O2 * 3.2 * math.exp(-550/TEMP)
                else:
                    raise UnprocessedRateCoef('Unprocessed reaction is found.')

#### Old calculation of the rate coefficients (for varing photolysis)

In [9]:
# #-----------------------------------------------------------------------------------------------------------------------------------#
# # FIRST IF MIGHT NEED CHANGES!
# if exp_name == 'init' or exp_name == 'emis':
#     # When J is not in comments, make {'num': 1}.
#     # The rates are calculated along with fluxes in a different notebook, because they are time dependent
#     for seq in sun_eqs.keys():
#         for eq in eqs:
#             if seq == eq['coef']:
#                 eq['num'] = 1
# #-----------------------------------------------------------------------------------------------------------------------------------#

In [10]:
# Store reactions where rate coefficient potentially has not been calculated
unprocessed_reactions = [eq for eq in eqs if 'num' not in eq.keys()]
class UnprocessedReaction(Exception):
    pass
if len(unprocessed_reactions) != 0:
    raise UnprocessedReaction('Rate coefficient is not calculated.')

In [11]:
# Save reaction descriptions as a json file
eqs_json_path = os.getcwd()
if not os.path.exists(eqs_json_path):
    os.makedirs(eqs_json_path)
with open(os.path.join(eqs_json_path, '{}_{}.json'.format(model_name, exp_name)), 'w') as f:
    json.dump(eqs, f)